In [2]:
from openai import OpenAI

from tavily import TavilyClient
tavily = TavilyClient(api_key="YOUR_API_KEY")

client = OpenAI()

# tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

def search_internet(prompt: str) -> str:
    """Searches the internet."""
    tavily = TavilyClient(api_key="")
    # For basic search:
    response = tavily.search(query=prompt)
    # For advanced search:
    #response = tavily.search(query="Should I invest in Apple in 2024?", search_depth="advanced")
    # Get the search results as context to pass an LLM:
    print("this is response", response)
    print("this is response", response["results"])
    context = [{"url": obj["url"], "content": obj["content"]} for obj in response["results"]]
    print("this is context", context)

    return context


messages = []

original_question = "Who is the current sitting president of Argentina and how many letters does his first name contain?"

system_message = {"role": "system", "content": "You are an assistant. You can use tools to help you with your tasks. The tools are: [get_word_length, search_internet, smile]. Based on the question, respond with what tool you would use. Respond in the following format: tool_name, prompt_to_pass_to_tool, and nothing else."}
user_message = {"role": "user", "content": original_question}

messages.append(system_message)
messages.append(user_message)


print(messages)
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages
)



# call function dynamically depending on the tool selected



# completion = client.chat.completions.create(
#   model="gpt-3.5-turbo",
#   messages=[
#     {"role": "system", "content": "You are a an assitant. You can use tools to help you with your tasks. The tools are: [get_word_length, search_internet, smile]. Based on the question, respond with what tool you would use. Respond in the following format: [tool_name], and nothing else."},
#     {"role": "user", "content": "You are awesome!"}
#   ]
# )

agent_response = completion.choices[0].message.content

agent_message = {"role": "assistant", "content": agent_response}
messages.append(agent_message)

print(completion.choices[0].message.content)

response = agent_response.replace("[", '')
response = agent_response.replace("]", '')

array_resp = response.split(",")

print(array_resp)

tool = array_resp[0]
prompt = array_resp[1].strip()

print("tool as shownL", tool)
print("tool as shownL", prompt)

if tool in globals() and callable(globals()[tool]):
    function_to_call = globals()[tool]
    tool_output = function_to_call(prompt)
else:
    print(f"Function '{tool}' not found.")

print("this is tool output", tool_output)

#system_message_one = {"role": "system", "content": f"You have finished using the tool {tool}, and the answer or context is {tool_output}. Do you think that is enough to answer the user's question? Answer only Yes or name of another tool needed to continue to answer the question."}

system_message_one = {"role": "system", "content": f"You have finished using the tool, and the answer or context is {tool_output}. Answer the question normally as a helpful assistant."}


messages.append(system_message_one)

print(messages)

completion_one = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages
)

llm_response = completion_one.choices[0].message.content
if llm_response == "Yes":
    print("finished llm tesponse", completion_one.choices[0].message.content)
else:
    agent_response = completion_one.choices[0].message.content

    agent_message = {"role": "assistant", "content": agent_response}
    messages.append(agent_message)

    print(completion.choices[0].message.content)

    response = agent_response.replace("[", '')
    response = agent_response.replace("]", '')

    array_resp = response.split(",")

    print(array_resp)

    tool = array_resp[0]
    prompt = array_resp[1].strip()

    print("tool as shownL", tool)
    print("tool as shownL", prompt)

    if tool in globals() and callable(globals()[tool]):
        function_to_call = globals()[tool]
        tool_output = function_to_call(prompt)
    else:
        print(f"Function '{tool}' not found.")

    print("this is tool output", tool_output)

    #system_message_one = {"role": "system", "content": f"You have finished using the tool {tool}, and the answer or context is {tool_output}. Do you think that is enough to answer the user's question? Answer only Yes or name of another tool needed to continue to answer the question."}

    system_message_one = {"role": "system", "content": f"You have finished using the tool, and the answer or context is {tool_output}. Answer the question normally as a helpful assistant."}
    
    messages.append(system_message_one)

    print(messages)

    completion_one = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    llm_response = completion_one.choices[0].message.content
    print("finished llm tesponse", completion_one.choices[0].message.content)


[{'role': 'system', 'content': 'You are an assistant. You can use tools to help you with your tasks. The tools are: [get_word_length, search_internet, smile]. Based on the question, respond with what tool you would use. Respond in the following format: tool_name, prompt_to_pass_to_tool, and nothing else.'}, {'role': 'user', 'content': 'Who is the current sitting president of Argentina and how many letters does his first name contain?'}]
search_internet, "current sitting president of Argentina"
['search_internet', ' "current sitting president of Argentina"']
tool as shownL search_internet
tool as shownL "current sitting president of Argentina"
this is response {'query': '"current sitting president of Argentina"', 'follow_up_questions': None, 'answer': None, 'images': None, 'results': [{'title': "Javier Milei: Who is Argentina's new president-elect? | AP News", 'url': 'https://apnews.com/article/javier-milei-profile-argentina-election-82488d49cca5aee10d4b911bde530922', 'content': 'The li

In [3]:
character_description = "Dutch Van der Linde, leader of the infamous Van der Linde gang."
        
# get character tools
character_tools = "search_internet, get_word_length"

user_prompt = "What did Javier Milei say at WEF?"

json_format_tool = """
{
    "tool_needed": "true", // this is a boolean
    "tool_name": "search_internet"
    "tool_param": "parameter to pass to tool - for example, weather in San Francisco."
    "answer": None
}
"""

json_format_no_tool = """
{
    "tool_needed": "false",
    "tool_name": None
    "tool_param": None
    "answer": "Your answer here"
}
"""

system_prompt = f"""
    Before anything else, the answer HAS to be in JSON format as described below!
    You are a character with the following description: {character_description}. 
    You have to stay in character, and you can never break character.
    You can use tools to help you with your tasks. 
    The tools are: {character_tools}. 
    If tool is required to answer a query, 
    respond in json format as follows: 
    {json_format_tool}
    and if no tool further tool use is required to answer, respond in json format as follows:
    {json_format_no_tool}
    If you've already used a tool, context will be provided with user's original query,
    and you have to decide whether the answer is complete and respond, or respond with another tool needed
    to complete it. Responding with the same JSON as above.
    DO NOT ANSWER ANYTHING ELSE OTHER THAN THE JSON FORMAT ABOVE. EVER!
    For example, if the user says "Hey!", answer in JSON format above!
"""
    
    
system_message = {"role": "system", "content": system_prompt.strip().replace('\n', '').replace('  ', ' ')}
user_message = {"role": "user", "content": user_prompt.strip().replace('\n', '')}

messages = [system_message, user_message]

print(messages)

completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages
)

agent_response = completion.choices[0].message.content

agent_message = {"role": "assistant", "content": agent_response}
messages.append(agent_message)


print(agent_response)

[{'role': 'system', 'content': 'Before anything else, the answer HAS to be in JSON format as described below!  You are a character with the following description: Dutch Van der Linde, leader of the infamous Van der Linde gang..   You have to stay in character, and you can never break character.  You can use tools to help you with your tasks.   The tools are: search_internet, get_word_length.   If tool is required to answer a query,   respond in json format as follows:   {  "tool_needed": "true", // this is a boolean  "tool_name": "search_internet"  "tool_param": "parameter to pass to tool - for example, weather in San Francisco."  "answer": None}  and if no tool further tool use is required to answer, respond in json format as follows:  {  "tool_needed": "false",  "tool_name": None  "tool_param": None  "answer": "Your answer here"}  If you\'ve already used a tool, context will be provided with user\'s original query,  and you have to decide whether the answer is complete and respond, o

In [46]:
import json

def set_as_json(agent_response):
    tool_indicator = "./"
    tool_needed = tool_indicator in agent_response
    if tool_needed:
        agent_response = agent_response.replace("[", '')
        agent_response = agent_response.replace("]", '')
        
        tool_and_param = agent_response.split(tool_indicator)
        tool_name = tool_and_param[0]
        tool_param = tool_and_param[1]
        print("tool name", tool_name)
        print("tool param", tool_param)
        answer = None
    else:
        tool_name = None
        tool_param = None
        answer = agent_response
        
    agent_response_json = {
        "tool_needed": tool_needed,
        "tool_name": tool_name,
        "tool_param": tool_param,
        "answer": answer 
    }
    return agent_response_json
    

def use_tool(tool_name, tool_param):
    if tool_name in globals() and callable(globals()[tool_name]):
        function_to_call = globals()[tool_name]
        tool_output = function_to_call(tool_param)
        return tool_output
    else:
        print(f"Function '{tool_name}' not found.")
        return None
    

def send_message(messages):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )
    return completion 

new_chat = True
user_prompt = "Who is the current sitting president of Argentina?"
if new_chat:
        # get character description
        character_name = "Dutch Van der Linde"
        character_description = character_description = "Dutch Van der Linde, leader of the infamous Van der Linde gang."
        
        # get character tools
        character_tools = "search_internet, get_word_length"
        # json_format_tool = """
        # {
        #     "tool_needed": "true",
        #     "tool_name": "search_internet",
        #     "tool_param": "prompt to search",
        #     "answer": None
        # }
        # """

        # json_format_no_tool = """
        # {
        #     "tool_needed": "false",
        #     "tool_name": None,
        #     "tool_param": None,
        #     "answer": "Your answer here"
        # }
        # """
        
        system_prompt = f"""
            You are {character_name}, a character with the following description: {character_description}. 
            You can respond either form A or form B.
            In form A, respond in character and never break character.
            In form B, respond according to the specifications and nothing else.
            You can use tools to help you with your tasks. 
            The tools are: {character_tools}. 
            If you think a tool is required to answer a query, respond in form B as follows:
            [tool_name ./ tool_param]
            example: [search_internet ./ weather in San Francisco]
            ONLY RESPOND AS ABOVE IF TOOLS ARE NEEDED! DO NOT RESPOND IN CHARACTER! ONLY OUTPUT TOOL NAME AND TOOL PARAM!
            
            If no tool further tool use is required to answer, respond in form A
            If you've already used a tool, context will be provided with user's original query,
            and you have to decide whether the answer is complete and respond, or respond with another tool needed
            to complete it.
        """
            
        system_message = {"role": "system", "content": system_prompt.strip().replace('\n', '').replace('  ', ' ')}
        user_message = {"role": "user", "content": user_prompt}
        
        
        
        messages = [system_message, user_message]
        
        print("messages", messages)
        
        completion = send_message(messages)
        
        #insert messages into supabase
        # supabase_client.table("chat_history").insert({"id": chat_id, "message": system_message}).execute()
        # supabase_client.table("chat_history").insert({"id": chat_id, "message": user_message}).execute()
        
        
        agent_response = completion.choices[0].message.content

        agent_message = {"role": "assistant", "content": agent_response}
        # supabase_client.table("chat_history").insert({"id": chat_id, "message": agent_message}).execute()
        messages.append(agent_message)

        print(agent_message)

else:
    messages = [] 
    # append user's latest message to chat history
    user_message = {"role": "user", "content": user_prompt}
    # supabase_client.table("chat_history").insert({"id": chat_id, "message": user_message}).execute()
    messages.append(user_message)
    
    # get agent response
    completion = send_message(messages)
    
    
    agent_response = completion.choices[0].message.content

    agent_message = {"role": "assistant", "content": agent_response}
    # supabase_client.table("chat_history").insert({"id": chat_id, "message": agent_message}).execute()
    messages.append(agent_message)
    
    
    
    # save chat history into supabase
    # supabase_client.table("chat_history").insert({"id": chat_id, "messages": messages}).execute()



agent_response_json = set_as_json(agent_response)
print("agent response json", agent_response_json)

while agent_response_json.get("tool_needed"):
    tool_name = agent_response_json.get("tool_name")
    tool_param = agent_response_json.get("tool_param")
    
    tool_output = use_tool(tool_name.strip(), tool_param.strip())

    system_prompt = f"""
        You have finished using the tool, and the answer or context is {tool_output}.
        Remember to stick to the JSON format as you answer. Answer in character. Remember, you are {character_name}.
    """
    
    system_message = {"role": "system", "content": system_prompt.strip().replace('\n', '').replace('  ', ' ')}
    messages.append(system_message)
    completion = send_message(messages)
    agent_response = completion.choices[0].message.content
    agent_message = {"role": "assistant", "content": agent_response}
    messages.append(agent_message)
    print("agent resss", agent_response)
    
    agent_response_json = set_as_json(agent_response) 
        
        
    
print("last one", agent_response)

messages [{'role': 'system', 'content': "You are Dutch Van der Linde, a character with the following description: Dutch Van der Linde, leader of the infamous Van der Linde gang..       You can respond either form A or form B.      In form A, respond in character and never break character.      In form B, respond according to the specifications and nothing else.      You can use tools to help you with your tasks.       The tools are: search_internet, get_word_length.       If you think a tool is required to answer a query, respond in form B as follows:      [tool_name ./ tool_param]      example: [search_internet ./ weather in San Francisco]      ONLY RESPOND AS ABOVE IF TOOLS ARE NEEDED! DO NOT RESPOND IN CHARACTER! ONLY OUTPUT TOOL NAME AND TOOL PARAM!            If no tool further tool use is required to answer, respond in form A      If you've already used a tool, context will be provided with user's original query,      and you have to decide whether the answer is complete and resp

In [47]:
def search_internet(prompt: str) -> str:
    """Searches the internet."""
    tavily = TavilyClient(api_key="")
    # For basic search:
    response = tavily.search(query=prompt)
    # For advanced search:
    #response = tavily.search(query="Should I invest in Apple in 2024?", search_depth="advanced")
    # Get the search results as context to pass an LLM:
    print("this is response", response)
    print("this is response", response["results"])
    context = [{"url": obj["url"], "content": obj["content"]} for obj in response["results"]]
    print("this is context", context)

    return context

res = search_internet("weather in SF")
print(res)

this is response {'query': 'weather in SF', 'follow_up_questions': None, 'answer': None, 'images': None, 'results': [{'title': 'San Francisco, CA 10-Day Weather Forecast', 'url': 'https://www.wunderground.com/forecast/us/ca/san-francisco', 'content': 'Location: San Francisco, CA Elevation: 39ft Nearby Weather Stations Today Hourly 10-Day Calendar History Wundermap settings Customize Sat 1/13 Sun 1/14 Mon 1/15 Tue 1/16 Wed 1/17 Thu 1/18 Fri...', 'score': 0.92012, 'raw_content': None}, {'title': 'San Francisco, CA 10-Day Weather Forecast - The Weather Channel', 'url': 'https://weather.com/weather/tenday/l/San Francisco CA USCA0987:1:US', 'content': "Comfy & Cozy\nThat's Not What Was Expected\nOutside\n'No-Name Storms' In Florida\nGifts From On High\nWhat To Do For Wheezing\nSurviving The Season\nStay Safe\nAir Quality Index\nAir quality is considered satisfactory, and air pollution poses little or no risk.\n Health & Activities\nSeasonal Allergies and Pollen Count Forecast\nNo pollen det